In [16]:
from src.SupertypeTools import CGCalcMat, dist_heatmap, SSE, hierarchical_cluster, Silhouette, correlation, Tuning_N, crop_mtx, SSE, triangle2square
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np

In [17]:
Nmin = 1
Nmax = 8

A_order = [
    ['A01_01', 'A26_01', 'A26_02', 'A26_03', 'A29_02', 'A30_01', 'A30_02', 'A30_03', 'A30_04', 'A32_01'],
    ['A02_01', 'A02_02', 'A02_03', 'A02_04', 'A02_05', 'A02_06', 'A02_07', 'A02_14', 'A02_17', 'A68_02', 'A69_01'],
    ['A03_01', 'A11_01', 'A31_01', 'A33_01', 'A33_03', 'A66_01', 'A68_01', 'A74_01'],
    ['A23_01', 'A24_02']
    ]

B_order = [
    ['B07_02','B07_03','B07_05','B15_08','B35_01','B35_03','B42_01','B51_01','B51_02','B51_03','B53_01','B54_01','B55_01','B55_02','B56_01','B67_01','B78_01'],
    ['B08_01','B08_02'],
    ['B14_02','B15_03','B15_09','B15_10','B15_18','B27_02','B27_03','B27_04','B27_05','B27_06','B27_07','B27_09','B38_01','B39_01','B39_02','B39_09','B48_01','B73_01'],
    ['B18_01','B37_01','B40_01','B40_02','B40_06','B44_02','B44_03','B45_01'],
    ['B15_16','B15_17','B57_01','B57_02','B58_01','B58_02'],
    ['B15_01','B15_02','B15_12','B15_13','B46_01','B52_01']
    ]

C_order = [
    ['C01_02','C03_02','C07_02','C08_01','C08_02','C12_02','C14_02','C16_01','C16_04'],
    ['C02_02','C03_07','C03_15','C04_01','C05_01','C06_02','C07_07','C12_04','C12_05','C15_02','C16_02','C17_01','C18_01']
    ]

BA_Mat_all = pd.read_csv("Computed_DistMtx/top10_matrix.csv", index_col=0)
BA_Mat_all = triangle2square(BA_Mat_all)

BA = crop_mtx(BA_Mat_all, A_order, flatten=True)
BB = crop_mtx(BA_Mat_all, B_order, flatten=True)
BC = crop_mtx(BA_Mat_all, C_order, flatten=True)


SSE_BA, _, _, _ = Tuning_N(crop_mtx(BA_Mat_all, A_order), Nmin, Nmax, None)
SSE_BB, _, _, _ = Tuning_N(crop_mtx(BA_Mat_all, B_order), Nmin, Nmax, None)
SSE_BC, _, _, _ = Tuning_N(crop_mtx(BA_Mat_all, C_order), Nmin, Nmax, None)

In [20]:
from sklearn.model_selection import ParameterGrid

SimMtx = "SM_THREAD_NORM"
# Grantham, PMBEC, SM_THREAD_NORM
grid = ParameterGrid({'sigma':[0.075, 0.1, 0.2, 0.3, 0.5],
                    'k':[0.8,1,2,4,8,16]})


for param in grid:

    sigma=param['sigma']
    k=param['k']
    print(sigma, k)

    # ======== HLA-A ===========
    AMat = CGCalcMat("HLA1_models/CG_DAT", SimMtx=SimMtx, AlleleListFile="Dataset_split/ReferencePanel-A.list", sigma=sigma, k=k)
    AMat = triangle2square(AMat)

    # SA = crop_mtx(AMat, A_order, flatten=True)
    # RA = correlation(SA, BA, show_plot=False)[2]
    _, SSE_SA, _, _ = Tuning_N(AMat, Nmin, Nmax, BA_Mat_all)
    # _, ASSE_BA3, _, _ = Tuning_N(AMat, 3, 3, BA_Mat_all)
    # _, ASSE_BA6, _, _ = Tuning_N(AMat, 6, 6, BA_Mat_all)

    # ========= HLA-B ===========
    BMat = CGCalcMat("HLA1_models/CG_DAT", SimMtx=SimMtx, AlleleListFile="Dataset_split/ReferencePanel-B.list", sigma=sigma,k=k)
    BMat = triangle2square(BMat)
    
    # SB = crop_mtx(BMat, B_order, flatten=True)
    # RB = correlation(SB, BB, show_plot=False)[2]
    _, SSE_SB, _, _ = Tuning_N(BMat, Nmin, Nmax, BA_Mat_all)
    # _, BSSE_BA3, _, _ = Tuning_N(BMat, 3, 3, BA_Mat_all)
    # _, BSSE_BA6, _, _ = Tuning_N(BMat, 6, 6, BA_Mat_all)

    # ========= HLA-C ===========
    CMat = CGCalcMat("HLA1_models/CG_DAT", SimMtx=SimMtx, AlleleListFile="Dataset_split/ReferencePanel-C.list", sigma=sigma, k=k)
    CMat = triangle2square(CMat)
    
    # SC = crop_mtx(CMat, C_order, flatten=True)
    # RC = correlation(SC, BC, show_plot=False)[2]
    _, SSE_SC, _, _ = Tuning_N(CMat, Nmin, Nmax, BA_Mat_all)
    # _, CSSE_BA, _, _ = Tuning_N(CMat, 2, 2, BA_Mat_all)

    
    # print(sum(np.array(SSE_SA) - np.array(SSE_BA)), sum(np.array(SSE_SB) - np.array(SSE_BB)), sum(np.array(SSE_SC) - np.array(SSE_BC)))
    print(sum(SSE_SA) + sum(SSE_SB) + sum(SSE_SC))
    # np.add(np.add(SSE_BA, SSE_BB), SSE_BC)
    # print(ASSE_BA3, ASSE_BA6, BSSE_BA3, BSSE_BA6, CSSE_BA)
    # print(RA, RB, RC, sum([RA, RB, RC]))

# Doytchinova SSE:

# Sidney SSE:

#6.16953559 3.6466475399999996 16.219613250000002 7.081748650000001 1.9410084

0.075 0.8
172.96945884
0.1 0.8
173.47616932
0.2 0.8
162.33695972
0.3 0.8
149.88879695999998
0.5 0.8
152.10624798000003
0.075 1
169.69292367999998
0.1 1
169.17306512
0.2 1
149.98585308
0.3 1
157.16223535999998
0.5 1
148.74224642000004
0.075 2
173.37475463999996
0.1 2
153.93090469999999
0.2 2
157.32278408
0.3 2
148.73454383
0.5 2
158.70105334
0.075 4
162.67645980000003
0.1 4
159.67068304
0.2 4
148.73454383
0.3 4
155.99240873999997
0.5 4
154.93854394
0.075 8
154.70182432000001
0.1 8
157.25733736
0.2 8
155.88178674
0.3 8
161.29208914
0.5 8
154.40580738
0.075 16
155.11638576000001
0.1 16
147.93162721000002
0.2 16
162.80702021
0.3 16
152.11241049999998
0.5 16
147.92834843999998
